In [2]:
import pandas as pd    #pandas: Python表格計算基本模組

train = pd.read_csv("D:/Data Mining/HW0/train.csv")    #pandas讀取kaggle下載的訓練資料集，訓練資料集含有特徵欄位與預測標籤 (survived column)
test = pd.read_csv("D:/Data Mining/HW0/test.csv")    #pandas讀取kaggle下載的測試資料集，測試資料集只含有特徵欄位

train.info() 
print("\n","="*60,"\n")
test.info()    #.info()查看訓練與測試集資訊

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass

In [3]:
def DataUniquePercentage(data):    #以百分比顯示各欄位有多少重複值
    return pd.DataFrame({'percentage_unique':data.apply(lambda x:x.unique().size/x.size*100)})

DataUniquePercentage(train)

,percentage_unique
PassengerId,100.000000
Survived,0.224467
Pclass,0.336700
Name,100.000000
Sex,0.224467
Age,9.988777
SibSp,0.785634
Parch,0.785634
Ticket,76.430976
Fare,27.833895


In [4]:
DataUniquePercentage(test)

,percentage_unique
PassengerId,100.000000
Pclass,0.717703
Name,100.000000
Sex,0.478469
Age,19.138756
SibSp,1.674641
Parch,1.913876
Ticket,86.842105
Fare,40.669856
Cabin,18.421053


In [5]:
import numpy as np    #numpy:Python數學計算基本模組

train['Age'] = train.loc[:,['Age']].fillna(np.nanmedian(np.array(train['Age'].tolist())))
test['Age'] = test.loc[:,['Age']].fillna(np.nanmedian(np.array(test['Age'].tolist())))
test['Fare'] = test.loc[:,['Fare']].fillna(np.nanmedian(np.array(test['Fare'].tolist())))

train = train.drop(['Cabin','PassengerId'],axis=1)
test = test.drop(['Cabin'],axis=1)
train = train.dropna(axis=0)
test = test.dropna(axis=0)

train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  889 non-null    int64  
 1   Pclass    889 non-null    int64  
 2   Name      889 non-null    object 
 3   Sex       889 non-null    object 
 4   Age       889 non-null    float64
 5   SibSp     889 non-null    int64  
 6   Parch     889 non-null    int64  
 7   Ticket    889 non-null    object 
 8   Fare      889 non-null    float64
 9   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 76.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Ag

In [6]:
from sklearn.preprocessing import LabelEncoder    #LabelEncoder是用來對分類型特徵值進行編碼，即對不連續的數值或文本進行編碼

le = LabelEncoder()
train['Name'] = le.fit_transform(train['Name'])
train['Sex'] = le.fit_transform(train['Sex'])
train['Ticket'] = le.fit_transform(train['Ticket'])
train['Embarked'] = le.fit_transform(train['Embarked'])

test['Name'] = le.fit_transform(test['Name'])
test['Sex'] = le.fit_transform(test['Sex'])
test['Ticket'] = le.fit_transform(test['Ticket'])
test['Embarked'] = le.fit_transform(test['Embarked'])

feature = train.loc[:,['Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Embarked']]
target = train.loc[:,['Survived']]


In [7]:
from sklearn.model_selection import train_test_split

feature_train, feature_val, target_train, target_val = train_test_split(feature, target, test_size=0.34)    #把訓練集資料分成訓練、驗證兩部分

In [8]:
from sklearn.linear_model import LogisticRegression

model_LR = LogisticRegression(random_state=0, solver='liblinear').fit(feature_train, target_train)

print(model_LR.score(feature_val, target_val))

0.801980198019802


C:\Users\Debra\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
from sklearn.tree import DecisionTreeClassifier

model_DT = DecisionTreeClassifier(random_state=0, criterion='entropy', max_depth=5).fit(feature_val, target_val)

print(model_DT.score(feature_val, target_val))

0.8976897689768977


In [10]:
passengerid = test['PassengerId'].tolist()
predict = model_DT.predict(test.loc[:, ['Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Embarked']])
df_dict = {'PassengerId': passengerid, 'Survived': predict}
submission = pd.DataFrame(data=df_dict)

submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB


In [18]:
submission.to_csv("D:/Data Mining/HW0/gender_submission.csv", index=False)

In [19]:
!jupyter nbconvert --to script HW0_109705013_賴郁忻.ipynb

[NbConvertApp] Converting notebook HW0_109705013_賴郁忻.ipynb to script
[NbConvertApp] Writing 2870 bytes to HW0_109705013_賴郁忻.py
